In [ ]:
import glob
from string import maketrans
from bs4 import BeautifulSoup
import urllib2
import logging
import telegram
from time import sleep
import random
import numpy as np

try:
    from urllib.error import URLError
except ImportError:
    from urllib2 import URLError  # python 2

class Corpus(object):
    def __init__(self):
        self.lines = []
        self.size = 0
    def Initialize(self, filename):
        next_line = True
        with open(filename) as f:
            for line in f:
                if len(line) > 1:
                    if not any(char.isdigit() for char in line):
                        if next_line:
                            self.lines.append(line.lower())
                            next_line = False
                        else:
                            self.lines[-1].join(line.lower())
                    else:
                        next_line = True

        self.size = len(self.lines)
    def GetAnswer(self, msg):
        there_is = np.zeros(self.size)
        maximum = 0
        index = -1
        split_symbols = ':;!?#%^"&*()|/-_=+@[]{}<>' + chr(0)
        destination_dictionary = '                          '
        tran_tab = maketrans(split_symbols, destination_dictionary)
        msg = msg.translate(tran_tab).lower().split()
        for i, line in enumerate(self.lines):
            for m in msg:
                if m in line:
                    there_is[i] += 1
                    if maximum < there_is[i]:
                        maximum = there_is[i]
                        index = i

        if index != -1:
            return self.lines[(index + 1)%self.size]
        else:
            return 'English, motherf**ker, do you speak it!'

answer = Corpus()
files = glob.glob('./subs/*')
for f in files:
     answer.Initialize(f)

def main():
    
    # Telegram Bot Authorization Token
    bot = telegram.Bot('137079651:AAGb_XiKJ2m9gj-ODYBmQwz8uKrRSfZc_fI')

    # get the first pending update_id, this is so we can skip over it in case
    # we get an "Unauthorized" exception.
    try:
        update_id = bot.getUpdates()[0].update_id
    except IndexError:
        update_id = None

    logging.basicConfig(
        format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

    while True:
        try:
            update_id = echo(bot, update_id)            
        except telegram.TelegramError as e:
            # These are network problems with Telegram.
            if e.message in ("Bad Gateway", "Timed out"):
                sleep(1)
            elif e.message == "Unauthorized":
                # The user has removed or blocked the bot.
                update_id += 1
            else:
                raise e
        except URLError as e:
            # These are network problems on our end.
            sleep(1)


def echo(bot, update_id):

    # Request updates after the last update_id
    for update in bot.getUpdates(offset=update_id, timeout=10):
        # chat_id is required to reply to any message
        chat_id = update.message.chat_id
        update_id = update.update_id + 1
        message = update.message.text
        if message:
            message = message.encode('utf-8')
            bot.sendMessage(chat_id=chat_id,
                            text=answer.GetAnswer(message))
            
    return update_id


def get_soup(url,header):
    return BeautifulSoup(urllib2.urlopen(urllib2.Request(url,headers=header)))

if __name__ == '__main__':
    main()